In [17]:
import numpy as np
import pandas as pd

from kmodes.kprototypes import KPrototypes
pd.options.display.max_rows = 150
pd.options.display.max_columns = 150

In [18]:
df = pd.read_csv("data/data_merge.csv")

In [19]:
dfMatrix = df.to_numpy()

In [20]:
catColumnsPos = [df.columns.get_loc(col) for col in list(df.select_dtypes('object').columns)]

In [21]:
kprototype = KPrototypes(n_jobs = -1, n_clusters = 15,n_init=4,max_iter=10, init = 'Huang', random_state = 0,verbose=10)
kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)

Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Init: initializing centroids
Init: initializing centroids
Init: initializing centroids
Starting iterations...
Init: initializing clusters
Init: initializing clusters
Init: initializing clusters
Starting iterations...
Init: initializing centroids
Init: initializing centroids
Init: initializing clusters
Init: initial

array([14, 14, 14, ...,  1,  8,  7], dtype=uint16)

In [22]:
# Add the cluster to the dataframe
df['Cluster Labels'] = kprototype.labels_
df['Segment'] = df['Cluster Labels'].map({0:'First', 1:'Second', 2:'Third', 3:'Fourth', 4:'Fifth', 5:'Sixth', 6:'Seventh', 7:'Eighth', 
 8:'Ninth', 9:'Tenth', 10:'Eleventh', 11:'Twelfth', 12:'Thirteenth', 13:'Fourteenth', 14:'Fifteenth'}
)
# Order the cluster
df['Segment'] = df['Segment'].astype('category')
df['Segment'] = df['Segment'].cat.reorder_categories(['First','Second','Third', 'Fourth', 'Fifth', 'Sixth', 'Seventh', 'Eighth', 'Ninth', 'Tenth',  'Eleventh', 'Twelfth', 'Thirteenth', 'Fourteenth', 'Fifteenth']
)

In [23]:
df

,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,review_score,review_creation_date,review_answer_timestamp,length_comment_title,length_comment_message,payment_type,payment_installments,payment_value,nb_items,sum_price,sum_freight_value,customer_city,customer_state,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,seller_city,seller_state,Cluster Labels,Segment
0,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,4.0,2017-10-11 00:00:00,2017-10-12 03:43:48,0.0,170.0,"credit_card,voucher",3.0,38.71,1.0,29.99,8.72,sao paulo,SP,268.0,4.0,500.0,19.0,8.0,13.0,housewares,maua,SP,14,Fifteenth
1,a20e8105f23924cd00833fd87daa0831,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,2017-08-18 14:44:43,2017-08-28 00:00:00,4.0,2017-08-19 00:00:00,2017-08-20 15:16:36,0.0,84.0,credit_card,3.0,37.77,1.0,29.99,7.78,sao paulo,SP,268.0,4.0,500.0,19.0,8.0,13.0,housewares,maua,SP,14,Fifteenth
2,26c7ac168e1433912a51b924fbd34d34,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,2017-08-07 18:30:01,2017-08-15 00:00:00,5.0,2017-08-08 00:00:00,2017-08-08 23:26:23,0.0,60.0,credit_card,1.0,37.77,1.0,29.99,7.78,sao paulo,SP,268.0,4.0,500.0,19.0,8.0,13.0,housewares,maua,SP,14,Fifteenth
3,53904ddbea91e1e92b2b3f1d09a7af86,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,2017-11-07 18:04:59,2017-11-13 00:00:00,3.0,2017-11-08 00:00:00,2017-11-10 19:52:38,0.0,0.0,boleto,1.0,44.09,1.0,29.99,14.10,florianopolis,SC,268.0,4.0,500.0,19.0,8.0,13.0,housewares,maua,SP,14,Fifteenth
4,ab8844663ae049fda8baf15fc928f47f,delivered,2017-08-10 13:35:55,2017-08-10 13:50:09,2017-08-11 13:52:35,2017-08-16 19:03:36,2017-08-23 00:00:00,5.0,2017-08-17 00:00:00,2017-08-21 12:43:27,0.0,41.0,credit_card,1.0,83.69,1.0,75.90,7.79,itaquaquecetuba,SP,398.0,3.0,238.0,20.0,10.0,15.0,baby,maua,SP,14,Fifteenth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100046,5a1470d43d8ad960d4199134d3df48e0,delivered,2018-08-10 21:14:35,2018-08-10 21:25:22,2018-08-13 13:54:00,2018-08-21 04:16:31,2018-08-30 00:00:00,5.0,2018-08-21 00:00:00,2018-08-21 22:01:55,0.0,0.0,credit_card,2.0,134.48,2.0,89.98,44.50,carai,MG,645.0,2.0,600.0,30.0,20.0,20.0,sports_leisure,paulo lopes,SC,2,Third
100047,a0b67404d84a70ef420a7f99ad6b190a,delivered,2018-07-01 10:23:10,2018-07-05 16:17:52,2018-07-04 14:34:00,2018-07-09 15:06:57,2018-07-20 00:00:00,5.0,2018-07-10 00:00:00,2018-07-10 18:32:29,0.0,0.0,boleto,1.0,93.13,1.0,79.00,14.13,ferraz de vasconcelos,SP,516.0,2.0,750.0,30.0,28.0,28.0,construction_tools_lights,porto ferreira,SP,2,Third
100048,d531d01affc2c55769f6b9ed410d8d3c,delivered,2018-07-24 09:46:27,2018-07-24 11:24:27,2018-07-24 15:14:00,2018-08-02 22:47:35,2018-08-16 00:00:00,5.0,2018-08-03 00:00:00,2018-08-04 11:22:40,0.0,0.0,debit_card,1.0,444.07,1.0,399.00,45.07,fortaleza,CE,729.0,2.0,2100.0,80.0,8.0,30.0,furniture_decor,americana,SP,1,Second
100049,9d8c06734fde9823ace11a4b5929b5a7,delivered,2018-05-22 21:13:21,2018-05-22 21:35:40,2018-05-24 12:28:00,2018-06-12 23:11:29,2018-06-08 00:00:00,4.0,2018-06-10 00:00:00,2018-06-13 09:17:47,0.0,0.0,credit_card,4.0,244.02,1.0,219.90,24.12,teofilo otoni,MG,531.0,1.0,5900.0,41.0,21.0,41.0,furniture_decor,sao paulo,SP,8,Ninth
